<a href="https://colab.research.google.com/github/kridtapon/ZLMA-Fisher-Fusion/blob/main/ZLMA_Fisher_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Zero Lag Moving Average (ZLMA)
def calculate_zlma(series, period=20):
    """
    Calculate Zero Lag Moving Average (ZLMA).
    """
    ema1 = series.ewm(span=period, adjust=False).mean()
    ema2 = ema1.ewm(span=period, adjust=False).mean()
    zlma = 2 * ema1 - ema2
    return zlma

# Function to calculate Fisher Transform and its Signal line
def calculate_fisher_transform(df, period=10):
    """
    Calculate Fisher Transform.
    """
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()

    # Avoid division by zero
    range_rolling = high_rolling - low_rolling
    range_rolling[range_rolling == 0] = np.nan  # Replace 0 with NaN to avoid errors

    # Calculate X
    X = 2 * ((df['Close'] - low_rolling) / range_rolling - 0.5)

    # Fisher Transform
    fisher = 0.5 * np.log((1 + X) / (1 - X))

    # Signal line (Exponential Moving Average of Fisher)
    fisher_signal = fisher.ewm(span=9, adjust=False).mean()

    return fisher, fisher_signal

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Zero Lag Moving Average (ZLMA)
df['ZLMA'] = calculate_zlma(df['Close'], period=20)

# Calculate Fisher Transform and Signal line
df['Fisher'], df['Fisher_Signal'] = calculate_fisher_transform(df, period=30)

# Define Entry and Exit signals based on ZLMA and Fisher Transform
df['Entry'] = (df['Close'] > df['ZLMA']) & (df['Fisher'] > 0)
df['Exit'] = (df['Close'] < df['ZLMA']) & (df['Fisher'] < 0)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           213168.954063
Total Return [%]                       113.168954
Benchmark Return [%]                   180.172876
Max Gross Exposure [%]                      100.0
Total Fees Paid                       9527.638475
Max Drawdown [%]                        69.438269
Max Drawdown Duration                       855.0
Total Trades                                   39
Total Closed Trades                            39
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            28.205128
Best Trade [%]                         150.472422
Worst Trade [%]                        -26.563713
Avg Winning Trade [%]                   25.245199
Avg Losing Trade [%]                    -4.294841
